In [5]:
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))
from src import fileutils
from src.paths import *
from reachml import ActionSet, ReachableSetDatabase
from scripts.utils import *

In [6]:
xgb_reach = fileutils.load(get_audit_results_file(** {
     "data_name": "fico",
     "action_set_name": "complex_nD",
     "model_type": "xgb",
     "method_name": "reach",
     }))

lr_reach = fileutils.load(get_audit_results_file(** {
     "data_name": "fico",
     "action_set_name": "complex_nD",
     "model_type": "logreg",
     "method_name": "reach",
     }))

rf_reach = fileutils.load(get_audit_results_file(** {
     "data_name": "fico",
     "action_set_name": "complex_nD",
     "model_type": "rf",
     "method_name": "reach",
     }))

In [7]:
action_set = fileutils.load(get_action_set_file("fico", action_set_name="complex_nD"))

db = ReachableSetDatabase(
        action_set=action_set,
        path=get_reachable_db_file(
                data_name="fico", action_set_name="complex_nD"
                ),
        )

In [8]:
data = fileutils.load(get_data_file("fico", action_set_name="complex_nD"))
stats = check_responsiveness(data=data, database=db)

In [9]:
stats

,y,yhat,yhat_post,recourse,n_reachable,n_feasible,complete,abstain
0,False,False,True,True,20480,1429,True,False
1,False,False,True,True,15488,235,True,False
2,False,True,True,True,4224,4214,True,False
3,False,False,False,False,12672,0,True,False
4,True,False,False,False,15552,0,True,False
...,...,...,...,...,...,...,...,...
5837,True,False,False,False,3872,0,True,False
5838,True,True,True,True,5632,5632,True,False
5839,True,True,True,True,6336,5790,True,False
5840,True,True,True,True,2816,2816,True,False


In [10]:
pred_to_str = lambda pred: "+" if pred else "-"

In [11]:
df = stats[["y", "n_reachable", "recourse", "abstain"]]
df = df.rename(columns={'y': 'label_type', 'n_reachable': 'region_size'})
df["label_type"] = df.apply(lambda row: pred_to_str(row["label_type"]), axis=1)
df["recourse"] = df.apply(lambda row: "feasible" if row["recourse"] else "infeasible", axis=1)
df

,label_type,region_size,recourse,abstain
0,-,20480,feasible,False
1,-,15488,feasible,False
2,-,4224,feasible,False
3,-,12672,infeasible,False
4,+,15552,infeasible,False
...,...,...,...,...
5837,+,3872,infeasible,False
5838,+,5632,feasible,False
5839,+,6336,feasible,False
5840,+,2816,feasible,False


In [12]:
lr_reach_df = pd.DataFrame(lr_reach.values())
xgb_reach_df = pd.DataFrame(xgb_reach.values())
rf_reach_df = pd.DataFrame(rf_reach.values())

In [13]:
lr_reach_df

,y_true,orig_prediction,post_prediction,a,found_a,actionable,abstains,certifies_infeasibility,recourse_exists
0,False,False,True,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",True,True,False,False,True
1,False,False,True,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",True,True,False,False,True
2,False,True,True,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",True,True,False,False,True
3,False,False,False,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",False,False,False,True,False
4,True,False,False,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",False,False,False,True,False
...,...,...,...,...,...,...,...,...,...
5837,True,False,False,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",False,False,False,True,False
5838,True,True,True,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",True,True,False,False,True
5839,True,True,True,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",True,True,False,False,True
5840,True,True,True,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",True,True,False,False,True


In [14]:
lr_df = df.copy()
lr_df["model_type"] = "lr"
lr_df["prediction_type"] = [pred_to_str(pred) for pred in lr_reach_df["orig_prediction"].values]
lr_df["correct"] = lr_reach_df["orig_prediction"] == lr_reach_df["y_true"]
lr_df = lr_df.sort_values(by=["region_size"])
lr_df["region_id"] = range(0, len(lr_df))
lr_df = lr_df[["label_type", "region_id", "model_type", "region_size", "prediction_type", "recourse", "correct", "abstain"]]
lr_df

,label_type,region_id,model_type,region_size,prediction_type,recourse,correct,abstain
1174,-,0,lr,2,-,infeasible,True,False
1450,+,1,lr,8,+,feasible,True,False
2008,+,2,lr,8,+,feasible,True,False
1811,-,3,lr,8,+,feasible,False,False
1610,+,4,lr,8,-,infeasible,False,False
...,...,...,...,...,...,...,...,...
3288,+,5837,lr,123904,-,feasible,False,False
5132,-,5838,lr,123904,-,feasible,True,False
3484,-,5839,lr,123904,-,feasible,True,False
2156,-,5840,lr,123904,-,feasible,True,False


In [15]:
xgb_df = df.copy()
xgb_df["model_type"] = "xgb"
xgb_df["prediction_type"] = [pred_to_str(pred) for pred in xgb_reach_df["orig_prediction"].values]
xgb_df["correct"] = xgb_reach_df["orig_prediction"] == xgb_reach_df["y_true"]
xgb_df = xgb_df.sort_values(by=["region_size"])
xgb_df["region_id"] = range(0, len(xgb_df))
xgb_df = xgb_df[["label_type", "region_id", "model_type", "region_size", "prediction_type", "recourse", "correct", "abstain"]]
xgb_df

,label_type,region_id,model_type,region_size,prediction_type,recourse,correct,abstain
1174,-,0,xgb,2,-,infeasible,True,False
1450,+,1,xgb,8,+,feasible,True,False
2008,+,2,xgb,8,+,feasible,True,False
1811,-,3,xgb,8,+,feasible,False,False
1610,+,4,xgb,8,-,infeasible,False,False
...,...,...,...,...,...,...,...,...
3288,+,5837,xgb,123904,-,feasible,False,False
5132,-,5838,xgb,123904,-,feasible,True,False
3484,-,5839,xgb,123904,-,feasible,True,False
2156,-,5840,xgb,123904,-,feasible,True,False


In [20]:
rf_df = df.copy()
rf_df["model_type"] = "rf"
rf_df["prediction_type"] = [pred_to_str(pred) for pred in rf_reach_df["orig_prediction"].values]
rf_df["correct"] = rf_reach_df["orig_prediction"] == rf_reach_df["y_true"]
rf_df = rf_df.sort_values(by=["region_size"])
rf_df["region_id"] = range(0, len(rf_df))
rf_df = rf_df[["label_type", "region_id", "model_type", "region_size", "prediction_type", "recourse", "correct", "abstain"]]
rf_df

,label_type,region_id,model_type,region_size,prediction_type,recourse,correct,abstain
1174,-,0,rf,2,-,infeasible,True,False
1450,+,1,rf,8,+,feasible,True,False
2008,+,2,rf,8,-,feasible,False,False
1811,-,3,rf,8,-,feasible,True,False
1610,+,4,rf,8,-,infeasible,False,False
...,...,...,...,...,...,...,...,...
3288,+,5837,rf,123904,-,feasible,False,False
5132,-,5838,rf,123904,-,feasible,True,False
3484,-,5839,rf,123904,-,feasible,True,False
2156,-,5840,rf,123904,-,feasible,True,False


In [22]:
barplot_df = pd.concat([lr_df, xgb_df, rf_df], axis=0)
# barplot_df
barplot_df.to_csv("/Users/avnikothari/Desktop/UCSD/Research/infeasible-recourse/paper/graphs/reachable_sets_barplot.csv")